<a href="https://colab.research.google.com/github/dannavon/PyCatan2/blob/master/CatanNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# from google.colab import drive
# drive.mount('/content/drive')


In [11]:
# %cd /content/drive/MyDrive/Git
# !pip install colored

In [12]:
# %cd /content/drive/MyDrive/Git/PyCatan2
# !git pull
# %cd /content/drive/MyDrive/Git/PyCatan2/src

In [13]:
import os.path

import matplotlib.pyplot as plt
from mcts import mcts_get_best_action
from mlp import MLP
from game import Game
from catan_wrp import Catan
import sys
from dataset import Dataset
import torch
from torch import Tensor
from training import MLPTrainer
import torch.nn.functional
from plot import plot_fit

In [14]:
hp_model_training = dict(loss_fn=torch.nn.MSELoss(),
                         batch_size=100,
                         num_epochs=100,
                         test_ratio=0.2,
                         valid_ratio=0.2,
                         early_stopping=100)

hp_optimizer = dict(lr=0.001,
                    weight_decay=0.01,
                    momentum=0.99)

hp_model = dict(hidden_layers_num=1,
                hidden_layers_size=20,
                activation='relu')

hp_mcts = dict(c=1,
               d=3,
               iterations_num=200)

hp_agent_training = dict(games_num=100)

In [15]:
def create_model(in_dim, out_dim, model_file):
    if os.path.isfile(model_file):
        print(f'loading model from "{model_file}"...')
        mlp = torch.load(model_file)
        print(mlp)
        return mlp

    mlp = MLP(
        in_dim=in_dim,
        dims=[hp_model['hidden_layers_size']] * hp_model['hidden_layers_num'] + [out_dim],
        nonlins=[hp_model['activation']] * hp_model['hidden_layers_num'] + ['none']
    )

    print('creating model...')
    print(mlp)
    return mlp


def train(dl_train, dl_valid, dl_test, model):
    loss_fn = hp_model_training['loss_fn']
    optimizer = torch.optim.SGD(params=model.parameters(), **hp_optimizer)
    trainer = MLPTrainer(model, loss_fn, optimizer)

    return trainer.fit(dl_train,
                       dl_valid,
                       num_epochs=hp_model_training['num_epochs'],
                       print_every=10,
                       early_stopping=hp_model_training['early_stopping'])

In [16]:
def train_agent(games_num, model_path):
    model = create_model(Catan.get_state_size(), Catan.get_players_num(), model_path)
    models = Catan.get_players_num()*[model]

    for i in range(1, games_num + 1):
        print(f'_________________game {i}/{games_num}________________')

        catan_game = Catan()

        ds = Dataset(hp_model_training['batch_size'], hp_model_training['valid_ratio'], hp_model_training['test_ratio'])

        turns_num = 0
        actions_num = 0

        while True:
            actions_num += 1

            best_action = mcts_get_best_action(catan_game, model, hp_mcts['c'], hp_mcts['d'], hp_mcts['iterations_num'])
            print("Player " + str(catan_game.get_turn() + 1) + ", action:" + str(best_action))

            reward = catan_game.make_action(best_action)
            if best_action[0] == 4:
                turns_num += 1

                print("Player " + str(catan_game.get_turn() + 1) + " turn!, dice: " + str(catan_game.dice))
                # print(catan_game.game.board)

            ds.add_sample(catan_game.get_state())
            if catan_game.is_over() or actions_num > 10000:
                print("Congratulations! Player %d wins!" % (catan_game.cur_id_player + 1))
                print("Final board:")
                print(catan_game.game.board)
                ds.set_label(reward)

                dl_train, dl_valid, dl_test = ds.get_data_loaders()
                fit_res = train(dl_train, dl_valid, dl_test, model)
                plot_fit(fit_res, log_loss=False, train_test_overlay=True)
                plt.show()
                print(ds)

                print(f'saving model in "{model_path}"')
                torch.save(model, model_path)
                break



In [ ]:
class Agent:
    def __init__(self, model, prune=True):
        self.model=model
        self.prune=prune


In [17]:
def test_agent(games_num):
    model1 = create_model(Catan.get_state_size(), Catan.get_players_num(), 'model')
    model2 = create_model(Catan.get_state_size(), Catan.get_players_num(), 'model2')
    agents = [Agent(model2), Agent(model2,prune=False), Agent(model1), Agent(model1,prune=False)]
    stats = {k: [] for k in range(Catan.get_players_num())}
    for i in range(1, games_num + 1):
        print(f'_________________game {i}/{games_num}________________')
        catan_game = Catan()
        actions_num = 0
        turns_num = 0
        while True:
            actions_num += 1

            best_action = mcts_get_best_action(catan_game, agents, hp_mcts['c'], hp_mcts['d'], hp_mcts['iterations_num'])
            print("Player " + str(catan_game.get_turn() + 1) + ", action:" + str(best_action))

            reward = catan_game.make_action(best_action)
            if best_action[0] == 4:
                turns_num += 1
                print("Player " + str(catan_game.get_turn() + 1) + " turn!, dice: " + str(catan_game.dice))
                # print(catan_game.game.board)

            if catan_game.is_over() or actions_num > 10000:
                stats[catan_game.cur_id_player].append([actions_num, int(turns_num/4)])
                print("Congratulations! Player %d wins!" % (catan_game.cur_id_player + 1))
                print("Final board:")
                print(catan_game.game.board)
                break
    return stats


In [18]:

if __name__ == '__main__':
    # train_agent(4, 'model2')
    print(test_agent(2))
    

creating model...
MLP(
  (mlp_layers): Sequential(
    (0): Linear(in_features=160, out_features=20, bias=True)
    (1): ReLU()
    (2): Linear(in_features=20, out_features=4, bias=True)
    (3): Identity()
  )
)
loading model from "model2"...
MLP(
  (mlp_layers): Sequential(
    (0): Linear(in_features=160, out_features=20, bias=True)
    (1): ReLU()
    (2): Linear(in_features=20, out_features=4, bias=True)
    (3): Identity()
  )
)
_________________game 1/2________________
Player 1, action:(<BuildingType.SETTLEMENT: 1>, (q: -2, r:0))
Player 1, action:(<BuildingType.ROAD: 0>, frozenset({(q: -2, r:0), (q: -3, r:1)}))
Player 2, action:(<BuildingType.SETTLEMENT: 1>, (q: -2, r:3))
Player 2, action:(<BuildingType.ROAD: 0>, frozenset({(q: -2, r:2), (q: -2, r:3)}))
Player 3, action:(<BuildingType.SETTLEMENT: 1>, (q: -4, r:3))
Player 3, action:(<BuildingType.ROAD: 0>, frozenset({(q: -5, r:3), (q: -4, r:3)}))
Player 4, action:(<BuildingType.SETTLEMENT: 1>, (q: 1, r:2))
Player 4, action:(<Buil

KeyboardInterrupt: 